In [1]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer

/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForCausalLM.from_pretrained("/data/mfx/huggingface/meta-llama/Llama-3.2-1B", device_map='cuda:0')
tokenizer = AutoTokenizer.from_pretrained("/data/mfx/huggingface/meta-llama/Llama-3.2-1B")

In [3]:
inputs = tokenizer("Write me a poem about the sea. The mother", return_tensors="pt").to("cuda:0")

In [4]:
output = model.generate(**inputs, max_new_tokens=128, do_sample=False)
print(tokenizer.batch_decode(output)[0])

/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|begin_of_text|>Write me a poem about the sea. The mother of all oceans, the sea is a place of mystery and wonder. It is a place of beauty and danger, a place of life and death. It is a place of love and hate, a place of peace and war. It is a place of hope and despair, a place of joy and sorrow. It is a place of life and death, a place of beauty and danger, a place of love and hate, a place of peace and war. It is a place of hope and despair, a place of joy and sorrow. It is a place of life and death, a place of beauty and danger, a place of love


In [5]:
lora_config = LoraConfig(r=32, target_modules=['q_proj'], pissaquant_config={'pissaquant_bits': 4}, init_lora_weights='PiSSAQuant')

In [6]:
peft_model = get_peft_model(model, lora_config)

tensor(0.0192, device='cuda:0')
tensor(0.0042, device='cuda:0')
tensor(0.0041, device='cuda:0')
tensor(0.0098, device='cuda:0')
tensor(0.0082, device='cuda:0')
tensor(0.0071, device='cuda:0')
tensor(0.0128, device='cuda:0')
tensor(0.0074, device='cuda:0')
tensor(0.0057, device='cuda:0')
tensor(0.0065, device='cuda:0')
tensor(0.0043, device='cuda:0')
tensor(0.0135, device='cuda:0')
tensor(0.0038, device='cuda:0')
tensor(0.0081, device='cuda:0')
tensor(0.0052, device='cuda:0')
tensor(0.0027, device='cuda:0')


In [7]:
peft_model.base_model.model.model.layers[0].self_attn.q_proj.lora_S.default

Parameter containing:
tensor([210.2891,  17.0796,  11.6353,   9.5772,   8.9909,   8.3452,   8.1227,
          8.0238,   7.9361,   7.7079,   7.5841,   7.1897,   7.1564,   7.0057,
          6.7993,   6.7189,   6.6833,   6.4882,   6.4230,   6.3611,   6.3275,
          6.1197,   6.0687,   6.0429,   5.9617,   5.8813,   5.8114,   5.7684,
          5.6735,   5.5499,   5.5069,   5.3306], device='cuda:0',
       requires_grad=True)

In [8]:
output = peft_model.generate(**inputs, max_new_tokens=128, do_sample=False)
print(tokenizer.batch_decode(output)[0])

/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|>Write me a poem about the sea. The mother of all oceans, the sea is a place of mystery and wonder. It is a place of beauty and danger, a place of life and death. It is a place of love and hate, a place of peace and war. It is a place of hope and despair, a place of joy and sorrow. It is a place of life and death, a place of beauty and danger, a place of love and hate, a place of peace and war. It is a place of hope and despair, a place of joy and sorrow. It is a place of life and death, a place of beauty and danger, a place of love


In [9]:
# model = peft_model.merge_and_unload()
# output = model.generate(**inputs, max_new_tokens=128, do_sample=False)
# print(tokenizer.batch_decode(output)[0])

In [10]:
peft_model.peft_config['default'].init_lora_weights = "PiSSAQuant_load"
peft_model.save_pretrained("PiSSAQuant-Llama-2-7B-r32/pissaquant_init")
model = peft_model.unload()
model.save_pretrained("PiSSAQuant-Llama-2-7B-r32")
tokenizer.save_pretrained("PiSSAQuant-Llama-2-7B-r32")

[2025-03-01 00:22:15,071] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data/mfx/miniconda3/envs/pissaquant/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/data/mfx/miniconda3/envs/pissaquant/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/pissaquant/lib/libcufile.so: undefined reference to `dlvsym'
/data/mfx/miniconda3/envs/pissaquant/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/pissaquant/lib/libcufile.so: undefined reference to `dlopen'
/data/mfx/miniconda3/envs/pissaquant/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/pissaquant/lib/libcufile.so: undefined reference to `dlclose'
/data/mfx/miniconda3/envs/pissaquant/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/pissaquant/lib/

('PiSSAQuant-Llama-2-7B-r32/tokenizer_config.json',
 'PiSSAQuant-Llama-2-7B-r32/special_tokens_map.json',
 'PiSSAQuant-Llama-2-7B-r32/tokenizer.json')

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("PiSSAQuant-Llama-2-7B-r32", device_map='cuda:0')
tokenizer = AutoTokenizer.from_pretrained("PiSSAQuant-Llama-2-7B-r32")
from peft import PeftModel
peft_model = PeftModel.from_pretrained(model, "PiSSAQuant-Llama-2-7B-r32/pissaquant_init")

{'base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight': tensor([[ 0.0216,  0.0216,  0.0216,  ...,  0.0228,  0.0228,  0.0228],
        [ 0.0282,  0.0282,  0.0282,  ..., -0.0144, -0.0144, -0.0144],
        [ 0.0250,  0.0250,  0.0250,  ...,  0.0203,  0.0203,  0.0203],
        ...,
        [ 0.0080,  0.0080,  0.0080,  ...,  0.0529,  0.0529,  0.0529],
        [-0.0093, -0.0093, -0.0093,  ...,  0.0196,  0.0196,  0.0196],
        [ 0.0427,  0.0427,  0.0427,  ..., -0.0285, -0.0285, -0.0285]],
       device='cuda:0'), 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight': tensor([[ 0.0074,  0.0028, -0.0009,  ...,  0.0062, -0.0042, -0.0014],
        [ 0.0148, -0.0168, -0.0028,  ..., -0.0015,  0.0144,  0.0046],
        [ 0.0176,  0.0014, -0.0191,  ...,  0.0121,  0.0201, -0.0006],
        ...,
        [ 0.0313, -0.0737,  0.0151,  ..., -0.0331, -0.0085,  0.0176],
        [ 0.0117,  0.0025, -0.0145,  ...,  0.0161, -0.0042,  0.0005],
        [ 0.0353, -0.0653,  0.0398,  ..., -0.

In [12]:
output = peft_model.generate(**inputs, max_new_tokens=128, do_sample=False)
print(tokenizer.batch_decode(output)[0])

/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|>Write me a poem about the sea. The mother of all oceans, the sea is a place of mystery and wonder. It is a place of beauty and danger, a place of life and death. It is a place of love and hate, a place of peace and war. It is a place of hope and despair, a place of joy and sorrow. It is a place of life and death, a place of beauty and danger, a place of love and hate, a place of peace and war. It is a place of hope and despair, a place of joy and sorrow. It is a place of life and death, a place of beauty and danger, a place of love
